In [1]:
import numpy as np
import pandas as pd

In [2]:
user_vendor_data=pd.read_csv("../r2Data/user_vendor_series_data.csv")

In [16]:
vendor_review_df=pd.read_json("../r2Data/yelp/yelp_academic_dataset_review.json",lines=True,nrows=600000)

In [3]:
vendor_business_info=pd.read_json("../r2Data/yelp/yelp_academic_dataset_business.json",lines=True)

In [4]:
vendor_business_info.sample(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
34215,Ut4tiRgbuL5sOa3hZemWGQ,Gold Dust West Casino Reno,444 Vine St,Reno,NV,89503,39.528924,-119.824745,3.0,125,1,{'GoodForKids': 'False'},"Casinos, Arts & Entertainment","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [19]:
vendor_review_df.shape

(600000, 9)

In [23]:
df_matrix=vendor_review_df.merge(vendor_business_info,on="business_id")

In [26]:
df_matrix['business_id'].duplicated().sum()

np.int64(585002)

In [27]:
df_matrix.sample(10)

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
487439,H0sjTpICb1T2AtpH3-jQxA,frkbB4jgmKzJaM5OqPiU6Q,L93Iw2rDhGT5dv3B86BTXg,5,0,0,1,If you are looking to get waxed Rita is the la...,2019-09-30 21:52:20,Tampa Sugaring,...,FL,33511,27.919124,-82.323760,4.5,7,1,"{'BikeParking': 'True', 'WiFi': 'u'free'', 'Bu...","Day Spas, Massage, Sugaring, Hair Removal, Wax...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-20:0', ..."
283101,4vWpAuj1gjnY3LaUhn_ekA,nj4wC4MJIhbHyFkZ_rd8fQ,avNUBfd-ZVCA0OkgEvaVqw,5,0,0,0,I used to go to Studio Branca for many years a...,2013-10-24 02:17:53,Studio Branca,...,MO,63141,38.677912,-90.464926,3.5,51,1,"{'BusinessParking': '{'garage': False, 'street...","Beauty & Spas, Nail Salons, Health & Medical, ...","{'Monday': '8:0-20:0', 'Tuesday': '7:0-21:0', ..."
392413,zIKfeAR0QjFlkftqXqCHmw,gxCLyxKrasqRZsv8qoRRfw,1skzWN0fxGgZNRE53Y_3gw,2,0,1,0,Perennial brews are the equivalent of Fizzie I...,2019-04-03 22:09:34,Perennial Artisan Ales,...,MO,63111,38.545541,-90.265343,4.0,125,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Breweries, Food","{'Wednesday': '15:0-20:0', 'Thursday': '15:0-2..."
499250,57-eri8PaRkq0gxbHI2afg,G6-THBuq3x_q3Kb106CJpw,AzseSGgDC6bVtMPEYo1CNQ,5,1,0,0,Creole Creamery has a huge variety of really c...,2016-06-21 22:07:52,Creole Creamery,...,LA,70115,29.924394,-90.108503,4.5,921,1,"{'BikeParking': 'True', 'GoodForKids': 'True',...","Food, Ice Cream & Frozen Yogurt","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'..."
558822,YyI8Og98lHFAeNCoNjylFg,Lcx7JDM5zHg1o27HbTggjg,z680Aylt8wN2KAeFM1hy-A,5,2,1,1,What can I say about Chatham Tap I think they ...,2021-04-05 14:36:19,Chatham Tap Mass. Ave.,...,IN,46204,39.776608,-86.146371,4.0,427,1,"{'Ambience': '{'romantic': False, 'intimate': ...","Bars, British, Pubs, Nightlife, Pizza, Restaur...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
78354,025ldYF8JaqYQ0bvlOw0HA,Jo-oqUCiaj_j3CSDolfdKw,aAdGIK9ewmiWtGzIcrQgNA,5,0,0,0,"Stop on my way to Orlando, the bartender and I...",2018-07-17 17:16:37,Gran Agave,...,FL,33578,27.894444,-82.318030,3.5,214,1,"{'RestaurantsReservations': 'False', 'Ambience...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
527444,8KHaMTjDWlZTA3LPDFEy-w,RPyg9Diz9KWUxn9iUM7Bfg,HPVpxcwDVjgZatc4M8bEnw,4,1,0,0,I travel to this area for business and always ...,2018-04-18 03:48:14,Frazer Diner,...,PA,19355,40.041407,-75.537486,3.5,51,1,"{'WiFi': 'u'no'', 'RestaurantsTakeOut': 'True'...","Restaurants, Breakfast & Brunch, Diners",None
321267,O6PVOarnonvX7_i7I2vUew,b_YyI6Q4DTXIlv41OITR-Q,8OPE0UaZHf7vdYkgbIj8vg,4,4,1,1,"""Packer Backer"" here... Have to say - love thi...",2015-01-11 19:58:02,Fox and Hound English Pub and Grille,...,IN,46250,39.904247,-86.087971,3.0,65,0,"{'RestaurantsTakeOut': 'True', 'RestaurantsDel...","American (New), Nightlife, Bars, Restaurants, ...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ..."
438024,7jwXqYpwE_XuJFbgoqR4ig,XrWAdRK4CUUK85Ak3x-HDw,Ms5xG8i4p80KSMcF3tt4ug,3,6,2,2,Meh.\nOnly went here for a bridal shower. The ...,2017-07-31 16:20:00,Sal DeForte's Ristorante,...,NJ,08628,40.266888,-74.805657,4.0,52,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Italian, Restaurants","{'Monday': '16:0-20:0', 'Tuesday': '16:0-20:0'..."
525781,NkNsmrEA8iss6vfyhu38Ug,fGIap6ckTYui9CZq9RxaCA,txasGoZ1qB5toVaFiTbpmQ,5,0,0,0,Great brunch spot - so glad we got to try it! ...,2021-07-13 00:09:01,The Brunchery,...,FL,33647,28.127905,-82.369112,4.0,88,1,"{'DogsAllowed': 'False', 'BusinessAcceptsCredi...","Restaurants, American (Traditional), Breakfast...","{'Monday': '8:0-14:0', 'Tuesday': '8:0-14:0', ..."


In [28]:
df_matrix.columns

Index(['review_id', 'user_id', 'business_id', 'stars_x', 'useful', 'funny',
       'cool', 'text', 'date', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars_y', 'review_count',
       'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

In [33]:
df_matrix.pivot_table(index = ['business_id'], aggfunc ='size') 

business_id
--ZVrH2X2QXBFdCilbirsw    31
--_9CAxgfXZmoFdNIRrhHA    11
--hF_3v1JmU9nlu4zfXJ8Q     3
-02xFuruu85XmDn2xiynJw    89
-0E7laYjwZxEAQPhFJXxow    16
                          ..
zzKVqizyl3QCb3GZmg_cNg     4
zzUxvYE-8Fj_dWqsPcagNg    10
zzjFdJwXuxBOGe9JeY_EMw    33
zztOG2cKm87I6Iw_tleZsQ     3
zzyx5x0Z7xXWWvWnZFuxlQ     8
Length: 14998, dtype: int64

# Vectorizing the revies of users

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize reviews
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
review_vectors = tfidf.fit_transform(df_matrix['text'])

In [56]:
review_vectors.shape[0]

600000

## Averaging reviews

In [43]:
# Map user_id to numeric index
user_id_map = {uid: idx for idx, uid in enumerate(df_matrix['user_id'].unique())}
df_matrix['user_idx'] = df_matrix['user_id'].map(user_id_map)




In [59]:
user_aggregation=[]
for i in range(review_vectors.shape[0]):
    user_aggregation.append(review_vectors[i].mean())

In [58]:
review_vectors[2].mean()

np.float64(0.0010833878206556902)

In [62]:
user_review_avg=pd.DataFrame(user_aggregation,columns=['user_review_avg'])

In [65]:
user_vendor_series=pd.concat([df_matrix,user_review_avg],axis=1)

In [66]:
user_vendor_series.to_csv("../r2Data/user_vendor_series_data.csv")

In [13]:
user_vendor_data.dropna(inplace=True)

# Convert vendor index to codes

In [17]:
# Map user_id to numeric index
user_id_map = {uid: idx for idx, uid in enumerate(user_vendor_data['business_id'].unique())}
user_vendor_data['vendor_idx'] = user_vendor_data['business_id'].map(user_id_map)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

# Vectorize user reviews
t1 = TfidfVectorizer(max_features=5000, stop_words='english')
review_vectors = t1.fit_transform(user_vendor_data['text'])




In [18]:
user_vendor_data['user_idx'].nunique()

347940

In [19]:
t2 = TfidfVectorizer(max_features=5000, stop_words='english')
vendor_profile = t2.fit_transform(user_vendor_data['categories'])

In [20]:
from sklearn.preprocessing import normalize

# Normalize user profiles and vendor profiles
user_profiles_normalized = normalize(review_vectors)
vendor_profiles_normalized = normalize(vendor_profile)


In [21]:
review_vectors.shape

(551582, 5000)

In [ ]:
num_users = user_vendor_data['user_idx'].nunique()
user_profiles = csr_matrix((num_users, review_vectors.shape[1]), dtype=np.float64)

for user_idx in range(num_users):
    user_reviews = review_vectors[np.array(user_vendor_data['user_idx'] == user_idx)]
    if user_reviews.shape[0] > 0:
        user_profiles[user_idx] = user_reviews.mean(axis=0)

print("User profiles shape:", user_profiles.shape)

C:\Users\ujwal\wedora_env\wedoraenv\Lib\site-packages\scipy\sparse\_index.py:197: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)
